# Imports

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

### Stopwords
Stopwords are words that are very common and add little meaning
examples: a, of, the

In [2]:
stopwords = nltk.corpus.stopwords.words('english')

### PorterStemmer
Words that have the same stem, typically have the same meaning
PorterStemmer cuts off the affixes so you just use the stem -> reduces word count (features)

In [3]:
ps = nltk.PorterStemmer()

# Read in Data

In [5]:
data = pd.read_csv("financial_news_sentiments_train.csv", encoding = "ISO-8859-1")
data.columns = ['sentiment', 'headlines']
data.head()

,sentiment,headlines
0,neutral,Technopolis plans to develop in stages an area...
1,negative,The international electronic industry company ...
2,positive,With the new production plant the company woul...
3,positive,According to the company 's updated strategy f...
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...


### Feature Creation
1. Punctuation percentage
2. Text Length

In [1]:
# Function to count punctuation
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

# Apply function
data['punct%'] = data['headlines'].apply(lambda x: count_punct(x))

NameError: name 'data' is not defined

In [ ]:
# Determine length of headline
data['text_len'] = data['headlines'].apply(lambda x: len(x) - x.count(" "))